# TorchVision 综合实践手册

> 本手册旨在系统梳理 `torchvision` 的核心能力，覆盖变换、模型、数据集、工具函数、算子、编解码与特征提取等模块。结合项目 `dataset/arch_linux` 提供的示例图像，通过代码演示帮助快速上手与对照查阅。

## 模块总览表

| 模块 | 典型入口 | 功能要点 | 主要用途 |
| --- | --- | --- | --- |
| `torchvision.transforms` / `transforms.v2` | `T.Compose`, `v2.RandomResizedCrop` | 图像/视频/目标的增广与预处理，V2 支持 `TVTensor` 与批处理 | 构建训练/推理的统一输入流水线 |
| `torchvision.datasets` | `datasets.CIFAR10`, `ImageFolder` | 内置经典数据集、文件夹读取与自定义基类 | 快速获得样本或包装本地数据 |
| `torchvision.models` | `resnet18`, `detection.fasterrcnn_resnet50_fpn` | 分类、检测、分割、视频、光流等预训练模型 | 迁移学习、特征抽取、端到端推理 |
| `torchvision.ops` | `box_convert`, `sigmoid_focal_loss` | 检测/分割常用算子、损失、层模块 | 自定义检测头、后处理 |
| `torchvision.utils` | `draw_bounding_boxes`, `save_image` | 可视化注释、保存图像网格 | 训练过程调试与结果展示 |
| `torchvision.io` | `read_image`, `encode_jpeg`, `decode_video` | 图像/视频的编解码与张量读取 | 构建高效的 IO / streaming 管道 |
| `torchvision.feature_extraction` | `create_feature_extractor` | 抽取中间特征用于可视化、蒸馏 | 模型解释、迁移学习 |
| `torchvision.tv_tensors` | `tv_t.Image`, `BoundingBoxes` | 携带元数据的增强张量类型 | 同步处理图像、框、关键点、掩码 |
| `torchvision.prototype`（可选） | `torchvision.prototype.transforms` | 实验性 API，跟踪前沿特性 | 评估新特性或未来 API 变更 |

In [ ]:
from pathlib import Path
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
import torchvision.transforms.v2 as v2
import torchvision.transforms.functional as F
import torchvision.tv_tensors as tv_t
from torchvision.utils import (
    draw_bounding_boxes,
    draw_segmentation_masks,
    draw_keypoints,
    flow_to_image,
    make_grid,
    save_image,
)
from torchvision.io import (
    read_image,
    decode_image,
    encode_jpeg,
    decode_video,
    write_png,
)
from torchvision.models import (
    resnet18,
    ResNet18_Weights,
    segmentation,
    detection,
    video as video_models,
    optical_flow,
    feature_extraction,
)
from torchvision import datasets
import matplotlib.pyplot as plt
from PIL import Image

os.environ.setdefault('OMP_NUM_THREADS', '1')
torch.set_num_threads(1)

#  示例数据集目录 : 目的是展示 torchvision 对图像的处理能力
DATASET_ROOT = Path('dataset/arch_linux')# 示例数据集目录
SAMPLE_IMAGE_PATH = sorted(DATASET_ROOT.glob('*.png'))[0]# 示例图像路径
base_pil = Image.open(SAMPLE_IMAGE_PATH).convert('RGB')# 转换为 RGB 模式
base_tensor = T.ToTensor()(base_pil) # 转换为 tensor 类型，范围 [0, 1]
base_uint8 = (base_tensor * 255).to(torch.uint8) # 转换为 uint8 类型，范围 [0, 255]

plt.rcParams['figure.figsize'] = (8, 4) # 图像显示大小
plt.rcParams['figure.facecolor'] = 'white' # 白色背景


# 可视化工具函数
def show_images(images, titles=None, cols=3, figsize=(12, 4)):
    """
    可视化多个图像，支持 PIL.Image 和 torch.Tensor。
    :param images: 图像列表，每个元素可以是 PIL.Image 或 torch.Tensor。
    :param titles: 图像标题列表，与 images 长度相同。
    :param cols: 每行显示的图像数量。
    :param figsize: 图像显示大小。
    """
    pil_images = []
    for img in images:
        if isinstance(img, Image.Image):
            pil_images.append(img)
        elif isinstance(img, torch.Tensor):
            tensor = img.detach().clone()
            if tensor.ndim == 4:
                tensor = make_grid(tensor)
            if tensor.dtype != torch.float32:
                tensor = tensor.float() / 255.0
            tensor = tensor.clamp(0, 1)
            pil_images.append(T.ToPILImage()(tensor))
        else:
            raise TypeError(f'不支持的图像类型: {type(img)}')
    total = len(pil_images)
    if total == 0:
        return
    cols = min(cols, total)
    rows = (total + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = np.array(axes).reshape(-1)
    titles = list(titles or [])
    if len(titles) < total:
        titles.extend([''] * (total - len(titles)))
    for idx, (ax, img, title) in enumerate(zip(axes, pil_images, titles)):
        ax.imshow(img)
        ax.axis('off')
        if title:
            ax.set_title(title)
    for ax in axes[total:]:
        ax.axis('off')
    plt.tight_layout()


def make_demo_video(num_frames: int = 4, size: int = 128) -> tv_t.Video:
    frames = []
    for i in range(num_frames):
        angle = i * 10
        frame = F.affine(base_tensor, angle=angle, translate=[0, 0], scale=1.0, shear=[0, 0])
        frames.append(frame)
    stacked = torch.stack(frames, dim=1)  # C, T, H, W
    return tv_t.Video(stacked)


def make_demo_boxes() -> tv_t.BoundingBoxes:
    boxes = torch.tensor([[50, 50, 300, 300], [150, 120, 420, 420]], dtype=torch.float32)
    return tv_t.BoundingBoxes(boxes, format=tv_t.BoundingBoxFormat.XYXY, canvas_size=base_tensor.shape[1:])


def make_demo_keypoints(num_points: int = 5) -> tv_t.Keypoints:
    xs = torch.linspace(80, 400, steps=num_points)
    ys = torch.linspace(90, 360, steps=num_points)
    keypoints = torch.stack([xs, ys], dim=1)
    return tv_t.Keypoints(keypoints, format='xy')


def describe_version():
    print('torchvision 版本:', torchvision.__version__)
    print('支持 transforms.v2:', hasattr(torchvision.transforms, 'v2'))

describe_version()

## Transforming images, videos, boxes and more 图像/视频/目标变换

本节概览 torchvision 变换体系，从输入约定、V1/V2 差异到性能注意事项，并通过 TVTensors 演示如何在现代 API 中同时处理图像、视频、目标框、关键点与掩码。

### Start here 入门指南
- **定位**：`torchvision.transforms` 提供数据增强与预处理工具，覆盖经典 `V1` API 以及推荐使用的 `V2` API。
- **核心思路**：将输入统一为张量或 `TVTensor`，以便组合操作并共享元数据（尺寸、坐标系等）。
- **V2 优势**：支持批处理、自动处理 `TVTensor` 元数据、与 `torch.compile` 更契合。

In [ ]:
# 使用 V2 随机组合同时处理图像、视频和边界框
reset_seed()
img = tv_t.Image(base_tensor.clone())
video = make_demo_video()
boxes = make_demo_boxes()

transform = v2.Compose([
    v2.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    v2.RandomHorizontalFlip(p=0.5),
])

img_out, boxes_out = transform(img, boxes)
video_out, _ = transform(video, boxes)  # 视频共享相同空间变换

print('图像输出类型:', type(img_out), img_out.shape)
print('边界框输出:', boxes_out)
print('视频输出形状:', video_out.shape)

### Supported input types and conventions 输入类型与约定
- **PIL / Tensor / TVTensor**：V1 支持 PIL 与张量，V2 在此基础上引入 `tv_tensors`，记录空间信息。
- **尺寸约定**：图像张量采用 `C×H×W`，视频采用 `C×T×H×W`，边界框使用 `(xmin, ymin, xmax, ymax)`。
- **批处理**：V2 支持批量维度，输入 `(B, C, H, W)` 时无需循环。

In [ ]:
# 统一输入类型的示例
batch = torch.stack([base_tensor, T.Resize((256, 256))(base_tensor)], dim=0)
print('批量维度:', batch.shape)
normalized = v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(batch)
print('归一化成功，数据范围:', normalized.amin().item(), normalized.amax().item())

### V1 or V2? 该如何选择
- **推荐**：新项目优先使用 `torchvision.transforms.v2`，享受类型安全与批处理能力。
- **兼容**：V1 仍广泛使用，可逐步迁移；`v2.functional` 提供与 V1 对齐的函数式接口。
- **混用策略**：通过 `v2.Compose([v2.ToImage(), v2.ToDtype(...)])` 作为入口，将旧有张量适配到 V2。

In [ ]:
# 将 V1 Compose 迁移到 V2 的示例
v1_pipeline = T.Compose([
    T.RandomResizedCrop(224),
    T.ColorJitter(0.2, 0.2, 0.2, 0.1),
    T.ToTensor(),
])

v2_pipeline = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(224),
    v2.ColorJitter(0.2, 0.2, 0.2, 0.1),
])

v1_out = v1_pipeline(base_pil)
v2_out = v2_pipeline(base_pil)
print('V1/V2 输出差异 (L1):', (v1_out - v2_out).abs().mean().item())

### Performance considerations 性能注意事项
- **避免重复 PIL ↔ Tensor 转换**：V2 `ToImage`/`ToDtype` 只需调用一次。
- **批量与并行**：结合 `torch.utils.data.DataLoader` 的 `batch_transforms` 提升吞吐。
- **GPU/CPU 切换**：`v2` 支持 `transform = transform.to(device)` 将参数移动到 GPU。

In [ ]:
# 将 V2 变换部署到 CUDA（若可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform_cuda = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]).to(device)

sample = base_tensor.to(device)
output = transform_cuda(sample)
print('输出设备:', output.device, '范围:', (output.min().item(), output.max().item()))

### Transform classes, functionals, and kernels 变换类型结构
- **Classes**：`v2.RandomCrop` 等面向对象接口，可在 `Compose` 中复用。
- **Functionals**：`v2.functional` 对应纯函数版本，适合自定义逻辑复用。
- **Kernels**：底层 C++/CUDA 算子实现，高性能且复用于 `torchvision.ops`。

In [ ]:
from torchvision.transforms import functional as F_v1
from torchvision.transforms import InterpolationMode

# V1/V2 函数式调用对比
rotated_v1 = F_v1.rotate(base_pil, angle=30, interpolation=InterpolationMode.BILINEAR)
rotated_v2 = v2.functional.rotate(base_tensor, angle=30)
show_images([rotated_v1, rotated_v2], ["V1 rotate", "V2 rotate"], cols=2)

### TorchScript support TorchScript 支持
- V2 变换遵循纯函数式设计，可通过 `torch.jit.script`。
- 自定义变换需将随机数使用 `torch.rand` 等 TorchScript 兼容 API。

In [ ]:
class ScriptableRandomInvert(v2.Transform):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, image):
        if torch.rand(1).item() < self.p:
            return 1.0 - image
        return image

script_transform = torch.jit.script(ScriptableRandomInvert())
print('TorchScript OK:', script_transform)

### V2 API reference - Recommended 推荐文档导航
- **核心入口**：`torchvision.transforms.v2` 模块文档覆盖类、函数式 API 与 `auto_augment` 策略。
- **补充**：`torchvision.tv_tensors` 文档描述各类张量包装及坐标格式。

### V1 API Reference 兼容文档提醒
- V1 文档仍在：`torchvision.transforms`。
- 查看历史项目时注意 `transforms.ToTensor`、`RandomCrop` 等默认行为差异（例如整型 -> float 的归一化）。

### TVTensors 生态
- **Image / Video**：携带尺寸信息，自动随变换更新。
- **KeyPoints**：支持 `xy`/`xyz` 格式，常用于姿态估计。
- **BoundingBoxes**：封装 `format`、`canvas_size`，简化坐标变换。
- **Mask**：继承 `TVTensor`，便于语义/实例分割掩码同步变换。
- **set_return_type / wrap**：控制自定义函数返回类型，保持 TVTensor 元信息。

In [ ]:
# TVTensor 组合示例
img = tv_t.Image(base_tensor.clone())
boxes = make_demo_boxes()
mask = tv_t.Mask(torch.zeros_like(base_tensor[0], dtype=torch.bool))
mask[..., 120:360, 160:400] = True
keypoints = make_demo_keypoints()

transform = v2.Compose([
    v2.RandomRotation(degrees=(-20, 20)),
    v2.RandomHorizontalFlip(p=1.0),
])

img_out, boxes_out, mask_out, keypoints_out = transform(img, boxes, mask, keypoints)
print('TVTensor 类型:', type(img_out), type(boxes_out), type(mask_out), type(keypoints_out))
print('关键点示例:', keypoints_out[:3])

In [ ]:
# 使用 set_return_type/wrap 创建自定义函数保持 TVTensor
@tv_t.wrap()
def brighten(image: tv_t.Image, delta: float = 0.1):
    return (image.float() + delta).clamp(0, 1)

bright_img = brighten(tv_t.Image(base_tensor), 0.2)
print('保持类型:', type(bright_img))

## Models and pre-trained weights 模型与预训练权重

本节总结 torchvision 模型库的加载方式与常见任务覆盖范围，并给出处理权重元数据、类别映射、推理与特征抽取的示例。

### General information on pre-trained weights 通用信息
- 预训练权重通过 `WeightsEnum` 管理，访问 `weights.meta` 获取类别、输入归一化等元数据。
- 初次加载权重需要网络，离线环境可使用 `weights.get_state_dict(progress=False)` 结合本地缓存。
- 建议在推理/评估阶段使用 `model.eval()` 并关闭梯度。

In [ ]:
weights = ResNet18_Weights.IMAGENET1K_V1
print('类别数:', len(weights.meta['categories']))
print('推荐输入归一化:', weights.meta['mean'], weights.meta['std'])

try:
    model = resnet18(weights=weights)
    print('成功加载预训练权重。')
except Exception as err:
    print('无法下载预训练权重，回退到随机初始化:', err)
    model = resnet18(weights=None)
model.eval()

### Classification 图像分类
- `torchvision.models` 提供 ResNet、ConvNeXt、ViT 等架构。
- 推理流程：加载权重 → 预处理 → 前向计算 → softmax。

In [ ]:
from torchvision.transforms.functional import resize

transform = v2.Compose([
    v2.Resize(ResNet18_Weights.DEFAULT.transforms().crop_size),
    v2.CenterCrop(ResNet18_Weights.DEFAULT.transforms().crop_size),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=ResNet18_Weights.DEFAULT.meta['mean'], std=ResNet18_Weights.DEFAULT.meta['std'])
])

sample = transform(base_pil).unsqueeze(0)
with torch.no_grad():
    logits = model(sample)
    probs = logits.softmax(dim=1)
    top5_prob, top5_catid = probs.topk(5)

for prob, catid in zip(top5_prob[0], top5_catid[0]):
    label = ResNet18_Weights.DEFAULT.meta['categories'][catid]
    print(f'{label}: {prob.item():.4f}')

### Semantic Segmentation 语义分割
- `torchvision.models.segmentation` 包含 DeepLabV3/FCN 等。
- 输出为 `dict`，需插值回原尺寸。

In [ ]:
seg_model = segmentation.deeplabv3_resnet50(weights=None).eval()
with torch.no_grad():
    seg_out = seg_model(base_tensor.unsqueeze(0))['out']
print('语义分割输出形状:', seg_out.shape)

### Object Detection, Instance Segmentation and Person Keypoint Detection 目标检测/实例分割/关键点
- `torchvision.models.detection` 提供 Faster R-CNN、Mask R-CNN、Keypoint R-CNN。
- 输入张量需为列表，输出同样是字典列表。

In [ ]:
det_model = detection.fasterrcnn_resnet50_fpn(weights=None).eval()
with torch.no_grad():
    det_out = det_model([base_tensor])
print('检测输出 keys:', det_out[0].keys())
print('预测框数量:', det_out[0]['boxes'].shape[0])

### Video Classification 视频分类
- `torchvision.models.video` 提供 R(2+1)D、MC3 等模型。
- 输入张量形状 `(B, C, T, H, W)`。

In [ ]:
video_model = video_models.r2plus1d_18(weights=None).eval()
video_clip = make_demo_video(num_frames=8, size=112).unsqueeze(0)  # B, C, T, H, W
with torch.no_grad():
    video_logits = video_model(video_clip)
print('视频分类输出维度:', video_logits.shape)

### Optical Flow 光流估计
- `torchvision.models.optical_flow` 提供 RAFT 系列。
- 输入两个连续帧，输出流场 `(B, 2, H, W)`。

In [ ]:
flow_model = optical_flow.raft_small(weights=None).eval()
pair = base_tensor.unsqueeze(0).repeat(2, 1, 1, 1)  # 两帧相同
with torch.no_grad():
    flow = flow_model(pair[0].unsqueeze(0), pair[1].unsqueeze(0))[0]
print('光流张量形状:', flow.shape)

## Datasets 数据集

- **Built-in datasets**：`torchvision.datasets` 提供 ImageNet、COCO、CIFAR 等。
- **Base classes for custom datasets**：`VisionDataset`、`ImageFolder`、`DatasetFolder` 简化自定义加载。
- **Transforms v2**：在 `datasets` 中通过 `transform=` 与 `target_transform=` 传入 V2 Pipeline。

In [ ]:
# 使用 ImageFolder 读取本地 arch_linux 图像
folder_dataset = datasets.ImageFolder(root='dataset', transform=v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]))
print('ImageFolder 类别:', folder_dataset.classes)
print('样本数量:', len(folder_dataset))

In [ ]:
# 使用 FakeData 构造训练样例，并演示 batch_transforms
train_transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.ColorJitter(0.2, 0.2, 0.2, 0.1),
])

fake_dataset = datasets.FakeData(size=4, image_size=(3, 224, 224), num_classes=10, transform=train_transforms)
images, labels = zip(*[fake_dataset[i] for i in range(4)])
show_images(images, [f'标签 {label}' for label in labels], cols=4, figsize=(12, 3))

## Utils 可视化与工具集

- `draw_bounding_boxes`、`draw_segmentation_masks`、`draw_keypoints`：用于快速调试目标任务。
- `flow_to_image`：将光流 `(2, H, W)` 转换为 RGB 可视化。
- `make_grid`、`save_image`：导出批量图像、保存结果。

In [ ]:
boxes = make_demo_boxes()
img_uint8 = (base_tensor * 255).to(torch.uint8)
annotated = draw_bounding_boxes(img_uint8, boxes.as_subclass(torch.Tensor), colors=['red', 'blue'], labels=['objectA', 'objectB'])
keypoints = make_demo_keypoints()
annotated_kp = draw_keypoints(annotated, keypoints.as_subclass(torch.Tensor).unsqueeze(0), colors='lime', radius=4)
show_images([img_uint8, annotated_kp], ['原始', '框+关键点'])

In [ ]:
mask = torch.zeros_like(base_uint8[0], dtype=torch.bool)
mask[120:360, 200:440] = True
mask_img = draw_segmentation_masks(base_uint8, mask.unsqueeze(0), colors=['magenta'], alpha=0.6)
show_images([mask_img], ['分割掩码叠加'])

In [ ]:
# 光流可视化
toy_flow = torch.stack(torch.meshgrid(
    torch.linspace(-1, 1, steps=base_tensor.shape[1]),
    torch.linspace(-1, 1, steps=base_tensor.shape[2]),
    indexing='ij'
), dim=0)
flow_vis = flow_to_image(toy_flow)
show_images([flow_vis], ['flow_to_image'])

In [ ]:
# make_grid + save_image 示例
batch = torch.stack([base_tensor, torch.flip(base_tensor, dims=[2])])
grid = make_grid(batch, nrow=2)
output_path = Path('souce_codeExplain/Transforms/output_grid.png')
output_path.parent.mkdir(parents=True, exist_ok=True)
save_image(batch, output_path)
show_images([grid], ['make_grid 预览'])
print('保存路径:', output_path)

## Operators 算子库

- **Detection/Segmentation Operators**：`torchvision.ops` 提供 NMS、RoIAlign、DeformConv2d 等。
- **Box Operators**：如 `box_convert`、`generalized_box_iou`。
- **Losses**：`sigmoid_focal_loss`、`complete_box_iou_loss`。
- **Layers**：`MLP`、`DeformConv2d`、`StochasticDepth`。

In [ ]:
from torchvision.ops import box_convert, generalized_box_iou, sigmoid_focal_loss, DeformConv2d

boxes_xyxy = torch.tensor([[50, 80, 200, 220], [120, 140, 260, 280]], dtype=torch.float32)
boxes_cxcywh = box_convert(boxes_xyxy, in_fmt='xyxy', out_fmt='cxcywh')
print('中心宽高格式:', boxes_cxcywh)

giou = generalized_box_iou(boxes_xyxy, boxes_xyxy)
print('广义 IoU:', giou)

logits = torch.randn(2, requires_grad=True)
target = torch.tensor([1.0, 0.0])
loss = sigmoid_focal_loss(logits, target)
print('Focal Loss:', loss.item())

layer = DeformConv2d(3, 8, kernel_size=3, padding=1)
input_feat = torch.rand(1, 3, 32, 32)
offset = torch.zeros(1, 18, 32, 32)  # 2*k*k*C_{out}
out_feat = layer(input_feat, offset)
print('DeformConv 输出形状:', out_feat.shape)

## Decoding / Encoding images and videos 图像/视频编解码
- `torchvision.io.decode_image`/`encode_jpeg`：直接在张量级别处理压缩数据。
- `torchvision.io.read_image`：读取为 `uint8` 张量，支持灰度/颜色模式。
- `torchvision.io.decode_video`：读取视频帧（注意旧 API 标记为 deprecated）。

In [ ]:
image_bytes = SAMPLE_IMAGE_PATH.read_bytes()
decoded = decode_image(torch.frombuffer(image_bytes, dtype=torch.uint8))
print('decode_image 输出:', decoded.shape, decoded.dtype)

encoded = encode_jpeg(decoded, quality=70)
print('重新编码 JPEG 大小:', encoded.size())

In [ ]:
loaded = read_image(str(SAMPLE_IMAGE_PATH))
print('read_image dtype:', loaded.dtype, 'range:', (loaded.min().item(), loaded.max().item()))
write_path = Path('souce_codeExplain/Transforms/reencoded.png')
write_png(loaded, write_path)
print('PNG 已写入:', write_path)

## IO operations 输入输出
- `torchvision.io` 提供读取/写入图像与视频的底层接口。
- 结合 `torch.utils.data.DataLoader` 可构建流式加载流水线。

## Video - DEPRECATED 视频旧接口
- 旧版 `torchvision.io.VideoClips` 与 `read_video` 部分接口标记为 deprecated。
- 建议迁移到新的视频 transforms 与 `torchvision.prototype`（若可用）。

## Feature extraction for model inspection 模型特征提取
- 通过 `torchvision.models.feature_extraction.create_feature_extractor` 获取中间层输出。
- 可用于可视化、蒸馏或特征分析。

In [ ]:
model_for_feature = resnet18(weights=None).eval()
return_nodes = {
    'layer1.1.relu': 'low_level',
    'layer4.1.relu': 'high_level',
}
feature_extractor = feature_extraction.create_feature_extractor(model_for_feature, return_nodes=return_nodes)
features = feature_extractor(base_tensor.unsqueeze(0))
for name, feat in features.items():
    print(name, feat.shape)

## API Reference 文档速查
- 官方文档入口：https://pytorch.org/vision/stable/
- 推荐结合 `help(torchvision.module)` 快速查看函数签名与 docstring。
- 保持本地/离线笔记，记录项目使用到的具体 API 组合与参数。